In [4]:
# Correct virtual environment
# Should be path/to/federated-learning-security/venv/bin/python
!which python3
# DLG algorithm thanks to: https://github.com/mit-han-lab/dlg

/Users/harrysharma/6CCS3PRJ/federated-learning-security/venv/bin/python3


In [5]:
import numpy as np
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
from torchvision import models, datasets, transforms
torch.manual_seed(50)

print(torch.__version__, torchvision.__version__)     

2.5.1 0.20.1


In [6]:
# Set-up image data to demonstrate gradient
image = Image.open("federated-learning-analysis/image/harry_ground_truth.JPG")
